In [1]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

C:\Users\Rajdeep\AppData\Local\conda\conda\envs\tfdeeplearning\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
batch_size = 64 # declaring the batch size

In [3]:
epochs = 100 # number of epochs to train

In [4]:
latent_dim = 256

In [5]:
num_samples = 1000

In [6]:
import os, sys

In [7]:
data_path = os.path.join(os.path.dirname(os.getcwd()), "datasets", "fra-eng", "fra.txt") # path to the corpus file
# note that it is super important to know the format of the corpus file
# in this case the format is such
# <eng-text><tab><french-translation of the same eng text>
# each new training example is in a new line

In [8]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [9]:
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split("\n") # spliting training examples

# remember we are trying to get only `sample` number of lines and not any more for training 
for line in lines[:min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split("\t")
    
    target_text = '\t' + target_text + "\n"
    
    input_texts.append(input_text)
    target_texts.append(target_text)
    
    # including chars in input and target characters
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [10]:
# here input_characters is the total number of input characters
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)


max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples: " + str(len(input_texts))) # total number of samples that we are training
print("Number of unique input tokens: " + str(num_encoder_tokens))
print("Number of unique output tokens: " + str(num_decoder_tokens))
print("Max sequence length for inputs: " + str(max_encoder_seq_length))
print("Max sequence length for outputs: " + str(max_decoder_seq_length))

Number of samples: 1000
Number of unique input tokens: 62
Number of unique output tokens: 77
Max sequence length for inputs: 11
Max sequence length for outputs: 33


In [11]:
# note that here we are only taking the first 1000 training examples

In [12]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
# recall that input_characters is a list of all the unique input characters that we have in the 1000 training examples
# that we have read so far

# what enumerate does is that it assigns a counter to the unique characters, so basically if we get the following list
# a = ["ola", "mundo", "keras"], we would get back from enumerate 1, "ola"; 2, "mundo"; 3, "keras"

target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [13]:
# here is the important part as we are defining what goes into the input data
# max_encoder seq length is the maximum number of characters in one training example in the entire corpus
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')

# the number of input_texts and the number of target_texts will be the same
decoder_input_data = np.zeros((len(target_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(target_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

# here we just created the vectors that we are going to use as inputs for the encoder and inputs and targets for the 
# decoder

In [14]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        # i is the first index as it is the ith training example 
        # t is the second index as the t-th character in ith training example
        encoder_input_data[i, t, input_token_index[char]] = 1
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1
        
        # Here is an important thing to understand
        ###############---------------------------IMPORTANT------------------------#####################
        # The decoder_target_data will be ahead of the decoder_input_data by one time step and this is an important factor
        # The decoder_target_data will obviously not contain the first character
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1
        

In [15]:
encoder_inputs = Input(shape=(None, num_encoder_tokens)) # num_encoder_tokens is essentially the vocab size

In [16]:
encoder = LSTM(latent_dim, return_state=True)
# Each unit or cell of LSTM within the layer has an internal cell state referred to as c and a hidden state referred
# to as h. 
# If we need to access the internal state c of the last time step we need to use return_state = True

In [17]:
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# here the encoder_outputs would be hidden state at last time step, state_h would be the hidden state of the last time step
# again and state_c is the internal state of the last time step

In [18]:
encoder_states = [state_h, state_c] # we will only keep the states and discard the output

In [19]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))

In [20]:
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
# return sequences will return the hidden states for each and every time step

In [21]:
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
# note here we are passing the encoder_states as the initial_state of the decoder lstm
# decoder_outputs will actially contain the hidden states of all the time steps as an array

In [22]:
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
# Dense is an ordinary dense neural network. 

In [23]:
decoder_outputs = decoder_dense(decoder_outputs)

In [24]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [25]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [26]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# passing all the data here

Train on 800 samples, validate on 200 samples
Epoch 1/100
800/800 [==============================] - 10s 12ms/step - loss: 1.6027 - val_loss: 1.6293
Epoch 2/100
800/800 [==============================] - 5s 6ms/step - loss: 1.3719 - val_loss: 1.6276
Epoch 3/100
800/800 [==============================] - 5s 6ms/step - loss: 1.3332 - val_loss: 1.5664
Epoch 4/100
800/800 [==============================] - 5s 6ms/step - loss: 1.3026 - val_loss: 1.5369
Epoch 5/100
800/800 [==============================] - 5s 6ms/step - loss: 1.2720 - val_loss: 1.5251
Epoch 6/100
800/800 [==============================] - 5s 6ms/step - loss: 1.2427 - val_loss: 1.5258
Epoch 7/100
800/800 [==============================] - 4s 6ms/step - loss: 1.2177 - val_loss: 1.5010
Epoch 8/100
800/800 [==============================] - 5s 6ms/step - loss: 1.1961 - val_loss: 1.4699
Epoch 9/100
800/800 [==============================] - 5s 6ms/step - loss: 1.1783 - val_loss: 1.4284
Epoch 10/100
800/800 [=====================

Epoch 81/100
800/800 [==============================] - 5s 6ms/step - loss: 0.2127 - val_loss: 1.1496
Epoch 82/100
800/800 [==============================] - 5s 6ms/step - loss: 0.2086 - val_loss: 1.1571
Epoch 83/100
800/800 [==============================] - 4s 5ms/step - loss: 0.2085 - val_loss: 1.1869
Epoch 84/100
800/800 [==============================] - 4s 6ms/step - loss: 0.1981 - val_loss: 1.1817
Epoch 85/100
800/800 [==============================] - 4s 6ms/step - loss: 0.1973 - val_loss: 1.2026
Epoch 86/100
800/800 [==============================] - 5s 7ms/step - loss: 0.1948 - val_loss: 1.1655
Epoch 87/100
800/800 [==============================] - 5s 6ms/step - loss: 0.1827 - val_loss: 1.1905
Epoch 88/100
800/800 [==============================] - 4s 5ms/step - loss: 0.1820 - val_loss: 1.2049
Epoch 89/100
800/800 [==============================] - 5s 6ms/step - loss: 0.1783 - val_loss: 1.2122
Epoch 90/100
800/800 [==============================] - 4s 5ms/step - loss: 0.1763

In [27]:
model.save('seq2seqKeras1000Samples.h5')

C:\Users\Rajdeep\AppData\Local\conda\conda\envs\tfdeeplearning\lib\site-packages\keras\engine\topology.py:2379: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


In [28]:
# Inference Model (Sampling sequences)
# For sampling the sequences here is the strategy to be followed

# 1. Encode the input and retrieve the initial encoder state
# 2. Run one step of the decoder with its initial encoder state and a "start of sequence" token as target
# 3. Output will be the next target token and current states

In [29]:
# In the original keras example, they are using the training examples themselves to predict the french statements
# However we would like to use predict new statements
# For this we would have to change a few things in order to achieve what we need to achieve

In [32]:
# preparing the test data
test_data_path = os.path.join(os.path.dirname(os.getcwd()), "datasets", "fra-eng", "fra-test.txt") # path to the test 
# corpus file
test_input_texts = []
with open(test_data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split("\n") # spliting training examples

# remember we are trying to get only `sample` number of lines and not any more for training 
for line in lines:
    input_text = line
    
#     target_text = '\t' + target_text + "\n"
    
    test_input_texts.append(input_text)
#     target_texts.append(target_text)
    
encoder_test_input_data = np.zeros((len(test_input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')

for i, (test_input_text) in enumerate(test_input_texts):
    print("i now:" + str(i))
    for t, char in enumerate(test_input_text):
        # i is the first index as it is the ith training example 
        # t is the second index as the t-th character in ith training example
#         print("t, char now: " + str(t) + ", " + str(char))
        encoder_test_input_data[i, t, input_token_index[char]] = 1

i now:0
i now:1
i now:2
i now:3
i now:4
i now:5
i now:6
i now:7
i now:8
i now:9


In [33]:
encoder_model = Model(encoder_inputs, encoder_states)

In [34]:
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_state_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_state_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_state_inputs, [decoder_outputs] + decoder_states)

In [35]:
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [36]:
def decode_sequence(input_seq):
    """
    This function is for decoding a sequence of english letters
    """
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq) # this is actually encoder_input data

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.
    
    # target_sequence is simply a one hot vector of size vocab size

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        # predicting one character at a time
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [40]:
for seq_index in range(10):
    # this trick below is just for converting input_seq to a list
    input_seq = encoder_test_input_data[seq_index: seq_index + 1] # we will need to form a new encoder_input_data
    decoded_sentence = decode_sequence(input_seq=input_seq)
    print('-')
    print("Input sentence: ", test_input_texts[seq_index])
    print("Decoded sentence: ", decoded_sentence)
    

-
Input sentence:  my drink
Decoded sentence:  Sois faice.

-
Input sentence:  quick brown
Decoded sentence:  J'es au fuui.

-
Input sentence:  How are you
Decoded sentence:  Comme c'est moi !

-
Input sentence:  Helloworld
Decoded sentence:  Silez ça !

-
Input sentence:  Well what
Decoded sentence:  Nous rompr !

-
Input sentence:  You know
Decoded sentence:  Qu la mes portir.

-
Input sentence:  Lets see
Decoded sentence:  Laissez-vous aller !

-
Input sentence:  Isha pali
Decoded sentence:  Este est houcees ti moi retre.

-
Input sentence:  network
Decoded sentence:  J'aidez-moi faifince.

-
Input sentence:  lets find
Decoded sentence:  Atendez coucie.

